In [1]:
import Pkg; Pkg.activate("..")

  Activating project at `~/com.github/lucifer1004/AdventOfCode.jl`


In [2]:
using AdventOfCode

In [3]:
year = 2022
day = 16

16

In [4]:
input = get_input(year, day)

"Valve AV has flow rate=0; tunnels lead to valves AX, PI\nValve JI has flow rate=0; tunnels lead to valves VD, HF\nValve FF has flow rate=0; tunnels lead to valves ZL, CG\nValve CG has flow rate=10; tunnels lead to valves TI, SU, RV, FF, QX\nValve RC has flow rate=18; tunnel" ⋯ 3053 bytes ⋯ "has flow rate=5; tunnels lead to valves TG, AV, HP\nValve XL has flow rate=0; tunnels lead to valves KU, VS\nValve AD has flow rate=0; tunnels lead to valves BK, RC\nValve EI has flow rate=0; tunnels lead to valves RV, AA\nValve OF has flow rate=19; tunnel leads to valve KV"

In [5]:
parsed_input = parse_input(input)

62-element Vector{SubString{String}}:
 "Valve AV has flow rate=0; tunnels lead to valves AX, PI"
 "Valve JI has flow rate=0; tunnels lead to valves VD, HF"
 "Valve FF has flow rate=0; tunnels lead to valves ZL, CG"
 "Valve CG has flow rate=10; tunnels lead to valves TI, SU, RV, FF, QX"
 "Valve RC has flow rate=18; tunnels lead to valves EQ, WR, AD"
 "Valve ZJ has flow rate=0; tunnels lead to valves GJ, WI"
 "Valve GJ has flow rate=21; tunnels lead to valves TG, YJ, EU, AZ, ZJ"
 "Valve VJ has flow rate=0; tunnels lead to valves UJ, AA"
 "Valve ER has flow rate=0; tunnels lead to valves QO, ZK"
 "Valve QO has flow rate=24; tunnels lead to valves MF, ER"
 "Valve LN has flow rate=0; tunnels lead to valves ZR, TI"
 "Valve SU has flow rate=0; tunnels lead to valves CG, LM"
 "Valve AJ has flow rate=12; tunnels lead to valves QX, JW, TR, MK"
 ⋮
 "Valve RO has flow rate=0; tunnels lead to valves AA, TC"
 "Valve TR has flow rate=0; tunnels lead to valves VD, AJ"
 "Valve VQ has flow rate=0; tunne

In [6]:
sample = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""

"Valve AA has flow rate=0; tunnels lead to valves DD, II, BB\nValve BB has flow rate=13; tunnels lead to valves CC, AA\nValve CC has flow rate=2; tunnels lead to valves DD, BB\nValve DD has flow rate=20; tunnels lead to valves CC, AA, EE\nValve EE has flow rate=3; tunnels lea" ⋯ 19 bytes ⋯ "Valve FF has flow rate=0; tunnels lead to valves EE, GG\nValve GG has flow rate=0; tunnels lead to valves FF, HH\nValve HH has flow rate=22; tunnel leads to valve GG\nValve II has flow rate=0; tunnels lead to valves AA, JJ\nValve JJ has flow rate=21; tunnel leads to valve II"

In [7]:
parsed_sample = parse_input(sample)

10-element Vector{SubString{String}}:
 "Valve AA has flow rate=0; tunnels lead to valves DD, II, BB"
 "Valve BB has flow rate=13; tunnels lead to valves CC, AA"
 "Valve CC has flow rate=2; tunnels lead to valves DD, BB"
 "Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE"
 "Valve EE has flow rate=3; tunnels lead to valves FF, DD"
 "Valve FF has flow rate=0; tunnels lead to valves EE, GG"
 "Valve GG has flow rate=0; tunnels lead to valves FF, HH"
 "Valve HH has flow rate=22; tunnel leads to valve GG"
 "Valve II has flow rate=0; tunnels lead to valves AA, JJ"
 "Valve JJ has flow rate=21; tunnel leads to valve II"

In [8]:
regex = r"Valve ([A-Z]{2}) has flow rate=(\d+); tunnels? leads? to valves? (.+)"

function preprocess(input)
    input = sort(input)
    mapping = Dict{String, Int}()
    n = length(input)
    valves = zeros(Int, n)
    adj = fill(100000000, n, n)
    foreach(enumerate(input)) do (i, line)
        u, _, _ = match(regex, line).captures
        mapping[u] = i
        adj[i, i] = 0
    end
    
    foreach(input) do line
        u, rate, vs = match(regex, line).captures
        u = mapping[u]
        valves[u] = parse(Int, rate)
        for v in split(vs, ", ")
            v = mapping[v]
            adj[u, v] = 1
        end
    end
    
    for k in 1:n
        for i in 1:n
            for j in 1:n
                adj[i, j] = min(adj[i, j], adj[i, k] + adj[k, j])
            end
        end
    end
    
    valid = [i for i in 1:n if valves[i] > 0 || mapping["AA"] == i]
    m = length(valid)
    adj2 = [adj[valid[i], valid[j]] for i in 1:m, j in 1:m]
    valves2 = [valves[i] for i in valid]
    return valves2, adj2
end

preprocess (generic function with 1 method)

In [9]:
function solver_pq(valves, adj, T)
    m = size(adj)[1]
    msk = (1 << (m - 1)) - 1
    best = OffsetArray(zeros(Int, 0:msk), 0:msk)
    pq = PriorityQueue()
    enqueue!(pq, (1, 0, 0), 0)
    while !isempty(pq)
        (u, t, state), now = peek(pq)
        now = -now
        dequeue!(pq)

        for v in 2:m
            i = v - 2
            if (state & (1 << i)) > 0
                continue
            end

            if t + adj[u, v] + 1 < T
                t1 = t + adj[u, v] + 1
                nxt = now + (T - t1) * valves[v]
                nxt_state = state | (1 << i)
                best[nxt_state] = max(best[nxt_state], nxt)
                nxt_key = (v, t1, nxt_state)
                if -get(pq, nxt_key, 1) < nxt
                    if haskey(pq, nxt_key)
                        delete!(pq, nxt_key)
                    end
                    enqueue!(pq, nxt_key, -nxt)
                end
            end
        end
    end
    return best
end

solver_pq (generic function with 1 method)

In [10]:
function solver_dp(valves, adj, T)
    m = length(valves)
    msk = (1 << (m - 1)) - 1
    dp = OffsetArray(fill(-1, 0:msk, m, T + 1), 0:msk, 1:m, 0:T)
    dp[0, 1, 0] = 0
    for t in 0:T - 1
        for u in 1:m
            for state in 0:msk
                now = dp[state, u, t]
                if now == -1
                    continue
                end

                for v in 2:m
                    i = v - 2
                    if (state & (1 << i)) > 0
                        continue
                    end

                    nxt = state | (1 << i)
                    t1 = t + adj[u, v] + 1
                    if t1 < T
                        dp[nxt, v, t1] = max(dp[nxt, v, t1], now + (T - t1) * valves[v])
                    end
                end
            end
        end
    end

    best = OffsetArray([maximum(dp[state, :, :]) for state in 0:msk], 0:msk)
    return best
end

solver_dp (generic function with 1 method)

In [11]:
function part_one(input, solver)
    valves, adj = preprocess(input)
    best = solver(valves, adj, 30)
    return maximum(best)
end

part_one (generic function with 1 method)

In [12]:
function part_two(input, solver)
    valves, adj = preprocess(input)
    best = solver(valves, adj, 26)
    m = length(valves)
    msk = (1 << (m - 1)) - 1
    for i in 0:msk
        j = (i - 1) & i
        while j > 0
            best[i] = max(best[i], best[j])
            j = (j - 1) & i
        end
    end
    return maximum(best[i] + best[msk⊻i] for i in 0:msk)
end

part_two (generic function with 1 method)

In [13]:
part_one(parsed_sample, solver_pq)

1651

In [14]:
part_one(parsed_sample, solver_dp)

1651

In [15]:
part_one(parsed_input, solver_pq)

1792

In [16]:
part_one_ans = part_one(parsed_input, solver_dp)

1792

In [17]:
submit_answer(year, day, part_one_ans)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 16 - Advent of Code 2022</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3355 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"

In [18]:
part_two(parsed_sample, solver_pq)

1707

In [19]:
part_two(parsed_sample, solver_dp)

1707

In [20]:
part_two(parsed_input, solver_pq)

2587

In [21]:
part_two_ans = part_two(parsed_input, solver_dp)

2587

In [22]:
submit_answer(year, day, part_two_ans, 2)

"<!DOCTYPE html>\n<html lang=\"en-us\">\n<head>\n<meta charset=\"utf-8\"/>\n<title>Day 16 - Advent of Code 2022</title>\n<!--[if lt IE 9]><script src=\"/static/html5.js\"></script><![endif]-->\n<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext'" ⋯ 3431 bytes ⋯ "Name(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)\n})(window,document,'script','//www.google-analytics.com/analytics.js','ga');\nga('create', 'UA-69522494-1', 'auto');\nga('set', 'anonymizeIp', true);\nga('send', 'pageview');\n</script>\n<!-- /ga -->\n</body>\n</html>"